In [1]:
import random
import string
import re
import pickle 
from unicodedata import normalize
import numpy
from numpy import array
from numpy.random import rand
from numpy.random import shuffle
from numpy import argmax
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import SimpleRNN,GRU,LSTM
from keras.layers import Dense, Embedding, RepeatVector, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_accuracy
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Fetching and decompressing the dataset 
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip

['Archive:  fra-eng.zip',
 'replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL',
 '(EOF or read error, treating as "[N]one" ...)']

In [3]:
# This function performs two tasks:

# 1. Loading the text data preserving the Unicode french characters and then
# 2. Each line of the text file contain English sentence and its French translation seperated by tab character. 

def loading_and_pairs(file):
    
	# opening the text file in read only mode with unicode encoding
	f = open(file = file, mode = 'rt', encoding = 'utf-8')
    
	# reading the text from the opened file
	text = f.read()
    
	# finally closing the file
	f.close()
    
    # Obtaining each line in the file
	l = text.strip().split('\n')
    
    # Obtaining the pairs of English sentence and its french translation
	pairs = [l_each.split('\t') for l_each in  l]
    
	return pairs   

In [4]:
# Cleaning the lines by removing all the non-printable characters, punctuation characters
# Given a list of lines cleaning them

def pairs_clean(lines_from_text):
    
	new_cleaned = list()
    
	# using regular expression for removing non-printable characters
	regular_non_print = re.compile('[^%s]' % re.escape(string.printable))
    
	# using regular expression for removing punctuation characters and obtaining translation table
	table = str.maketrans('', '', string.punctuation)
    
	for pair in lines_from_text:
        
		clean_pair = list()
		for new_line in pair:
            
			# normalization to remove canonical and compatibility related issues
			new_line = normalize('NFD', new_line).encode('ascii', 'ignore')
			new_line = new_line.decode('UTF-8')
            
			# tokenizing the white space
			new_line = new_line.split()
            
			# normalizing the text to lowercase
			new_line = [word.lower() for word in new_line]
            
			# removing punctuation from each token using regular expression table 
			new_line = [word.translate(table) for word in new_line]
            
			# removing non-printable characters using the above regular expression
			new_line = [regular_non_print.sub('', w) for w in new_line]
            
			# removing the non-alphabetic tokens such as numbers
			new_line = [word for word in new_line if word.isalpha()]
            
			# store as string
			clean_pair.append(' '.join(new_line))
		new_cleaned.append(clean_pair)
	return array(new_cleaned)

In [5]:
# saving the list of clean sentences to file
def saving_data(sentences, file):
	pickle.dump( sentences, open(file = file, mode = 'wb'))

In [6]:
# loading the text dataset
text_file = 'fra.txt'
text_pairs = loading_and_pairs(text_file)

In [7]:
# cleaning the sentences
cleaned_pairs = pairs_clean(text_pairs)

In [8]:
# saving the cleaned pairs to file
saving_data(cleaned_pairs, 'eng-fre.pkl')

In [9]:
# Total number of translation sentences
print("There a total of {} pairs of tranlations in the dataset".format(cleaned_pairs.shape[0]))

There a total of 190206 pairs of tranlations in the dataset


In [10]:
# Looking at few sentences

random_sample_list = random.sample(range(0, cleaned_pairs.shape[0]), 30)
for i in random_sample_list:
	print('English : %s\nFrench : %s \n' % (cleaned_pairs[i,0], cleaned_pairs[i,1]))

English : weve studied french for several years
French : nous avons etudie le francais pendant plusieurs annees 

English : i go there every year
French : jy vais tous les ans 

English : i want to thank you for your time
French : je veux vous remercier pour votre temps 

English : please help yourself to the cookies
French : veuillez vous servir des biscuits 

English : this dictionary is mine
French : ce dictionnaire est a moi 

English : i dont think i can help you
French : je ne pense pas pouvoir vous aider 

English : im willing to risk that
French : je suis pret a prendre ce risque 

English : please show me the menu
French : montrezmoi le menu sil vous plait 

English : how did you get that
French : dou tienstu ceci 

English : i dont believe it happened that way
French : je ne crois pas que ca se soit passe de cette maniere 

English : when did you start dating
French : quand avezvous commence a sortir ensemble 

English : dublin is in ireland
French : dublin est en irlande 

E

In [11]:
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle

In [12]:
# loading the previously cleaned data 
def loading_sentences(filename):
	return pickle.load(open(filename, 'rb'))

In [13]:
# loading the raw dataset
text_dataset = loading_sentences('eng-fre.pkl')

In [14]:
# There are over 190000 pairs of sentences
# It will take long time for training and testing the model
# Hence dataset size is reduced
num_sentences = 15000 #clean_pairs.shape[0]
reduced_dataset_15000 = text_dataset[:num_sentences, :]
train_size = numpy.rint(0.7 * num_sentences)
validation_size = numpy.rint(0.1 * num_sentences)
test_size = numpy.rint(0.2 * num_sentences)

# randomly shuffling the dataset
shuffle(reduced_dataset_15000)

# spliting the reduced dataset into train, validation and test
split_1 = int(train_size)
split_2 = int(train_size+validation_size)
split_3 = int(train_size+validation_size+test_size)
train, validation, test = reduced_dataset_15000[:split_1], reduced_dataset_15000[split_1:split_2], reduced_dataset_15000[split_2:split_3]

In [15]:
train.shape, validation.shape, test.shape

((10500, 3), (1500, 3), (3000, 3))

In [16]:
# saving the reduced dataset to training, validation and testing data

saving_data(text_dataset, 'eng-fre-total.pkl')
saving_data(train, 'eng-fre-train.pkl')
saving_data(validation, 'eng-fre-validation.pkl')
saving_data(test, 'eng-fre-test.pkl')

In [17]:
# Using keras tokenize class, for mapping the words to integers needed for modeling

def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

In [18]:
# Obtaining the maximum sentence length from the list of phrases 

def max_length(input_lines):
	return max(len(each_line.split()) for each_line in input_lines)

In [19]:
# encoding and padding the input and output sequences

def encode_sequences(tokenizer, length, lines):
	# Each input and output sequence are  encoded to integers
	text_encoded_integers = tokenizer.texts_to_sequences(lines)
	# Obtained sequences are padded with 0 values at the end to make their lenght as maxmim phrase length
	padding_sequences = pad_sequences(text_encoded_integers, maxlen=length, padding='post')
	return padding_sequences

In [20]:
# The output target sequences (English sentences) has to be one hot encoded as the model will 
# predicts probability of each word in the vocabulary as output. 

def encode_output(output_sequences, vocabulary_size):
	output_list = list()
	for output_sequence in output_sequences:
		cat = to_categorical(output_sequence, num_classes=vocabulary_size)
		output_list.append(cat)
	encoded_output = array(output_list)
	encoded_output = encoded_output.reshape(output_sequences.shape[0], output_sequences.shape[1], vocabulary_size)
	return encoded_output

In [21]:
# reverse mapping an predicted sequence of integers to a words by looking up tokenizer

def reverse_mapping(output_integer, tokenizer):
	for w, i in tokenizer.word_index.items():
		if i == output_integer:
			return w
	return None

In [22]:
# mapping the sequence of integers for generating string of words

def predict_sequence(rnn_model, tokenizer, original_data):
	prediction = rnn_model.predict(original_data, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target_list = list()
	for i in integers:
		word = reverse_mapping(i, tokenizer)
		if word is None:
			break
		target_list.append(word)
	return ' '.join(target_list)

In [23]:
# Evaluating the performance of each model using BLEU score by comparing predicted result to original/expected sequences 


def model_evaluation(model_name, tokenizer_used, sources_text, raw_text_dataset):
	actual_value, predicted_value = list(), list()
	bleu_scores = []
	for i, j in enumerate(sources_text):
		# translating the encoded input text sequence
		j = j.reshape((1, j.shape[0]))
		translation = predict_sequence(model_name, eng_tokenizer, j)
		raw_target, raw_src = raw_text_dataset[i][0], raw_text_dataset[i][1]
        
        # Printing 50 French to English translations by the model
        
		if i < 50:
			print('French(Source) : %s\nTarget : %s\nPredicted : %s \n' % (raw_src, raw_target, translation))
        
        
		actual_value.append([raw_target.split()])
		predicted_value.append(translation.split())
        
    # Calculating BLEU score
	bleu_score = corpus_bleu(actual_value, predicted_value, weights=(0.25, 0.25, 0.25, 0.25))
    
	# Print BLEU score
	print('BLEU score: %f' % bleu_score)
    
	return bleu_score

In [24]:
# loading the total, train, validation and test datasets 
dataset = loading_sentences('eng-fre-total.pkl')
train = loading_sentences('eng-fre-train.pkl')
valid = loading_sentences('eng-fre-validation.pkl')
test = loading_sentences('eng-fre-test.pkl')

In [25]:
dataset.shape

(190206, 3)

In [26]:
reduced_dataset_15000.shape

(15000, 3)

In [27]:
# Building the French tokenizer
french_tokenizer = create_tokenizer(reduced_dataset_15000[:, 1])
french_maximum_length = max_length(reduced_dataset_15000[:, 1])
french_vocabulary_size = len(french_tokenizer.word_index) + 1

In [28]:
print('French Vocabulary Size:', french_vocabulary_size)
print('French Maximum Sentence Length:', french_maximum_length)

French Vocabulary Size: 5753
French Maximum Sentence Length: 10


In [29]:
# Building the English tokenizer
eng_tokenizer = create_tokenizer(reduced_dataset_15000[:, 0])
english_maximum_length = max_length(reduced_dataset_15000[:, 0])
english_vocabulary_size = len(eng_tokenizer.word_index) + 1

In [30]:
print('English Vocabulary Size:',english_vocabulary_size)
print('English Maximum Sentence Length:', english_maximum_length)

English Vocabulary Size: 2776
English Maximum Sentence Length: 5


In [31]:
# Word index of English tokenizer
eng_tokenizer.word_index

{'i': 1,
 'it': 2,
 'you': 3,
 'tom': 4,
 'im': 5,
 'a': 6,
 'is': 7,
 'me': 8,
 'its': 9,
 'he': 10,
 'was': 11,
 'youre': 12,
 'are': 13,
 'go': 14,
 'we': 15,
 'be': 16,
 'to': 17,
 'that': 18,
 'this': 19,
 'were': 20,
 'dont': 21,
 'the': 22,
 'do': 23,
 'get': 24,
 'not': 25,
 'can': 26,
 'ill': 27,
 'have': 28,
 'up': 29,
 'they': 30,
 'my': 31,
 'no': 32,
 'here': 33,
 'did': 34,
 'come': 35,
 'in': 36,
 'need': 37,
 'like': 38,
 'she': 39,
 'your': 40,
 'let': 41,
 'out': 42,
 'all': 43,
 'love': 44,
 'thats': 45,
 'him': 46,
 'take': 47,
 'want': 48,
 'us': 49,
 'got': 50,
 'keep': 51,
 'hes': 52,
 'help': 53,
 'am': 54,
 'one': 55,
 'stop': 56,
 'lets': 57,
 'on': 58,
 'theyre': 59,
 'toms': 60,
 'look': 61,
 'how': 62,
 'who': 63,
 'stay': 64,
 'well': 65,
 'see': 66,
 'please': 67,
 'saw': 68,
 'what': 69,
 'try': 70,
 'must': 71,
 'back': 72,
 'home': 73,
 'just': 74,
 'so': 75,
 'know': 76,
 'lost': 77,
 'now': 78,
 'cant': 79,
 'leave': 80,
 'feel': 81,
 'had': 82,
 'to

In [32]:
# Word index of French tokenizer
french_tokenizer.word_index

{'je': 1,
 'tom': 2,
 'suis': 3,
 'a': 4,
 'pas': 5,
 'nous': 6,
 'il': 7,
 'cest': 8,
 'vous': 9,
 'jai': 10,
 'est': 11,
 'de': 12,
 'ne': 13,
 'le': 14,
 'la': 15,
 'un': 16,
 'me': 17,
 'en': 18,
 'ca': 19,
 'tu': 20,
 'les': 21,
 'une': 22,
 'que': 23,
 'estce': 24,
 'sommes': 25,
 'etes': 26,
 'qui': 27,
 'es': 28,
 'fait': 29,
 'elle': 30,
 'sont': 31,
 'ils': 32,
 'tout': 33,
 'ce': 34,
 'va': 35,
 'des': 36,
 'bien': 37,
 'soyez': 38,
 'ma': 39,
 'du': 40,
 'besoin': 41,
 'elles': 42,
 'ete': 43,
 'te': 44,
 'y': 45,
 'faut': 46,
 'ici': 47,
 'etesvous': 48,
 'jaime': 49,
 'ai': 50,
 'mon': 51,
 'se': 52,
 'sois': 53,
 'nest': 54,
 'veux': 55,
 'fais': 56,
 'aller': 57,
 'moi': 58,
 'jetais': 59,
 'etait': 60,
 'cela': 61,
 'lai': 62,
 'on': 63,
 'lair': 64,
 'partir': 65,
 'train': 66,
 'peux': 67,
 'au': 68,
 'personne': 69,
 'faire': 70,
 'avons': 71,
 'cetait': 72,
 'sest': 73,
 'reste': 74,
 'estu': 75,
 'sens': 76,
 'mal': 77,
 'deteste': 78,
 'toi': 79,
 'maintenant': 8

In [33]:
# Building the training dataset
x_train = encode_sequences(french_tokenizer, french_maximum_length, train[:, 1])
y_train = encode_sequences(eng_tokenizer, english_maximum_length, train[:, 0])
y_train = encode_output(y_train, english_vocabulary_size)

In [34]:
# Building the validation dataset
x_valid = encode_sequences(french_tokenizer, french_maximum_length, valid[:, 1])
y_valid = encode_sequences(eng_tokenizer, english_maximum_length, valid[:, 0])
y_valid = encode_output(y_valid, english_vocabulary_size)

In [35]:
# Building testing dataset
x_test = encode_sequences(french_tokenizer, french_maximum_length, test[:, 1])
y_test = encode_sequences(eng_tokenizer, english_maximum_length, test[:, 0])
y_test = encode_output(y_test, english_vocabulary_size)

In [36]:
# Model 1: Both Encoder GRU and Decoder GRU cell
def both_gru(french_vocab, english_vocab, french_timesteps, english_timesteps, num_units):
	learning_rate = 1e-2
	model = Sequential()
	model.add(Embedding(french_vocab, num_units, input_length = french_timesteps, mask_zero = True))
	model.add(GRU(num_units))
	model.add(RepeatVector(english_timesteps))
	model.add(GRU(num_units,return_sequences=True))
	model.add(TimeDistributed(Dense(english_vocab, activation='softmax')))
	return model

In [37]:
# Defining the Model-1

learning_rate = 1e-2
both_gru = both_gru(french_vocabulary_size, english_vocabulary_size, french_maximum_length, english_maximum_length, 256)
both_gru.compile(optimizer = Adam(learning_rate), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [38]:
# Summary of the Model-1

print(both_gru.summary())
plot_model(both_gru, to_file='gru_model.png', show_shapes=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 256)           1472768   
_________________________________________________________________
gru (GRU)                    (None, 256)               394752    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 5, 256)            0         
_________________________________________________________________
gru_1 (GRU)                  (None, 5, 256)            394752    
_________________________________________________________________
time_distributed (TimeDistri (None, 5, 2776)           713432    
Total params: 2,975,704
Trainable params: 2,975,704
Non-trainable params: 0
_________________________________________________________________
None
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.g

In [39]:
# Fitting the Model-1
filename = 'both_gru.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
history_both_gru = both_gru.fit(x_train, y_train, epochs = 30, batch_size = 64, validation_data=(x_valid, y_valid), callbacks = [checkpoint], verbose = 2)

Epoch 1/30
165/165 - 27s - loss: 3.6164 - categorical_accuracy: 0.4724 - val_loss: 3.1550 - val_categorical_accuracy: 0.5169

Epoch 00001: val_loss improved from inf to 3.15496, saving model to both_gru.h5
Epoch 2/30
165/165 - 11s - loss: 2.7978 - categorical_accuracy: 0.5495 - val_loss: 2.7806 - val_categorical_accuracy: 0.5655

Epoch 00002: val_loss improved from 3.15496 to 2.78064, saving model to both_gru.h5
Epoch 3/30
165/165 - 10s - loss: 2.3569 - categorical_accuracy: 0.5946 - val_loss: 2.5868 - val_categorical_accuracy: 0.5869

Epoch 00003: val_loss improved from 2.78064 to 2.58683, saving model to both_gru.h5
Epoch 4/30
165/165 - 10s - loss: 2.0226 - categorical_accuracy: 0.6243 - val_loss: 2.4399 - val_categorical_accuracy: 0.6047

Epoch 00004: val_loss improved from 2.58683 to 2.43994, saving model to both_gru.h5
Epoch 5/30
165/165 - 10s - loss: 1.7371 - categorical_accuracy: 0.6542 - val_loss: 2.3341 - val_categorical_accuracy: 0.6193

Epoch 00005: val_loss improved from 2.

In [40]:
# Saving the history of the Model-1

df_history_both_gru = pd.DataFrame(history_both_gru.history)
df_history_both_gru.to_csv('df_history_both_gru.csv')
print(df_history_both_gru)

        loss  categorical_accuracy  val_loss  val_categorical_accuracy
0   3.616443              0.472419  3.154955                  0.516933
1   2.797794              0.549467  2.780640                  0.565467
2   2.356876              0.594590  2.586829                  0.586933
3   2.022557              0.624267  2.439938                  0.604667
4   1.737110              0.654209  2.334076                  0.619333
5   1.498589              0.680933  2.267290                  0.627867
6   1.308935              0.707505  2.218283                  0.635067
7   1.136943              0.731181  2.184579                  0.640800
8   0.981828              0.758438  2.148189                  0.650000
9   0.868323              0.780362  2.183507                  0.651333
10  0.790319              0.797029  2.220069                  0.654800
11  0.731204              0.809181  2.222612                  0.650133
12  0.676890              0.820933  2.208533                  0.652533
13  0.

In [41]:
# Loading the Model-1
both_gru = load_model('both_gru.h5')

In [42]:
#  Model-1 on the training sequences

bleu_train_both_gru = model_evaluation(both_gru, eng_tokenizer, x_train, train)

French(Source) : je me suis marre
Target : i had some fun
Predicted : i had i fun 

French(Source) : laissezmoi sortir
Target : let me out
Predicted : let me out 

French(Source) : on peut me faire confiance
Target : im trustworthy
Predicted : we must win 

French(Source) : quelquun estil la
Target : anybody home
Predicted : anybody anybody 

French(Source) : mettezle la
Target : put it there
Predicted : put it there 

French(Source) : vous etes sournois
Target : youre sneaky
Predicted : youre sneaky 

French(Source) : ne vous precipitez pas
Target : dont rush
Predicted : dont back 

French(Source) : degage
Target : go away
Predicted : get away 

French(Source) : elle laide
Target : she helps him
Predicted : she helps him 

French(Source) : je ny parviens pas
Target : i cant do it
Predicted : i cant it it 

French(Source) : jai la priorite sur vous
Target : i outrank you
Predicted : i outrank you 

French(Source) : je men suis doute
Target : i thought so
Predicted : i thought so 

Fren

In [43]:
# Testing Model-1 

bleu_test_both_gru = model_evaluation(both_gru, eng_tokenizer, x_test, test)

French(Source) : faites comme il vous plaira
Target : do as you like
Predicted : do as you want 

French(Source) : nous partons a la retraite
Target : were retiring
Predicted : we got 

French(Source) : tes givree
Target : youre nuts
Predicted : youre stink 

French(Source) : apportez un appareil photo
Target : bring a camera
Predicted : bring your camera 

French(Source) : tu sembles contrariee
Target : you seem upset
Predicted : you seem upset 

French(Source) : peutetre viendratelle
Target : she may come
Predicted : how could us 

French(Source) : je dispose dun plan
Target : i have a plan
Predicted : i had a plan 

French(Source) : je dormis tout le jour
Target : i slept all day
Predicted : i from from 

French(Source) : jai vu cela
Target : i saw that
Predicted : i said that 

French(Source) : tom sauta
Target : tom jumped
Predicted : tom was 

French(Source) : pouvonsnous taider
Target : can we help you
Predicted : can you help 

French(Source) : voila cinq dollars
Target : heres

In [44]:
# Model 2: Both Encoder LSTM and Decoder LSTM cell
def both_lstm(french_vocab, english_vocab, french_timesteps, english_timesteps, num_units):
	learning_rate = 1e-2
	model = Sequential()
	model.add(Embedding(french_vocab, num_units, input_length= french_timesteps, mask_zero=True))
	model.add(LSTM(num_units))
	model.add(RepeatVector(english_timesteps))
	model.add(LSTM(num_units,return_sequences=True))
	model.add(TimeDistributed(Dense(english_vocab, activation='softmax')))
	return model

In [45]:
# Defining Model-2

learning_rate = 1e-2
both_lstm = both_lstm(french_vocabulary_size, english_vocabulary_size, french_maximum_length, english_maximum_length, 256)
both_lstm.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [46]:
# Summarising Model-2

print(both_lstm.summary())
plot_model(both_lstm, to_file='both_lstm.png', show_shapes=True)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 256)           1472768   
_________________________________________________________________
lstm (LSTM)                  (None, 256)               525312    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 256)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 256)            525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 2776)           713432    
Total params: 3,236,824
Trainable params: 3,236,824
Non-trainable params: 0
_________________________________________________________________
None
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz

In [47]:
# Fitting the Model-2

filename = 'both_lstm.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
history_both_lstm = both_lstm.fit(x_train, y_train, epochs=30, batch_size=64, validation_data=(x_valid, y_valid), callbacks=[checkpoint], verbose=2)

Epoch 1/30
165/165 - 22s - loss: 3.6130 - categorical_accuracy: 0.4722 - val_loss: 3.1232 - val_categorical_accuracy: 0.5061

Epoch 00001: val_loss improved from inf to 3.12317, saving model to both_lstm.h5
Epoch 2/30
165/165 - 14s - loss: 2.7115 - categorical_accuracy: 0.5495 - val_loss: 2.6402 - val_categorical_accuracy: 0.5712

Epoch 00002: val_loss improved from 3.12317 to 2.64020, saving model to both_lstm.h5
Epoch 3/30
165/165 - 14s - loss: 2.1778 - categorical_accuracy: 0.6073 - val_loss: 2.4160 - val_categorical_accuracy: 0.5972

Epoch 00003: val_loss improved from 2.64020 to 2.41602, saving model to both_lstm.h5
Epoch 4/30
165/165 - 15s - loss: 1.8106 - categorical_accuracy: 0.6439 - val_loss: 2.2561 - val_categorical_accuracy: 0.6215

Epoch 00004: val_loss improved from 2.41602 to 2.25609, saving model to both_lstm.h5
Epoch 5/30
165/165 - 16s - loss: 1.5008 - categorical_accuracy: 0.6817 - val_loss: 2.1887 - val_categorical_accuracy: 0.6320

Epoch 00005: val_loss improved fro

In [48]:
# Saving the history of the Model-2

df_history_both_lstm = pd.DataFrame(history_both_lstm.history)
df_history_both_lstm
df_history_both_lstm.to_csv('df_history_both_lstm.csv')

In [49]:
# Loading the Model-2
both_lstm = load_model('both_lstm.h5')

In [50]:
# Model-2 training sequences
bleu_train_both_lstm = model_evaluation(both_lstm, eng_tokenizer, x_train, train)

French(Source) : je me suis marre
Target : i had some fun
Predicted : i had doubts fun 

French(Source) : laissezmoi sortir
Target : let me out
Predicted : let me out 

French(Source) : on peut me faire confiance
Target : im trustworthy
Predicted : i trustworthy me 

French(Source) : quelquun estil la
Target : anybody home
Predicted : anybody tom home 

French(Source) : mettezle la
Target : put it there
Predicted : put it there 

French(Source) : vous etes sournois
Target : youre sneaky
Predicted : youre sneaky 

French(Source) : ne vous precipitez pas
Target : dont rush
Predicted : dont rush 

French(Source) : degage
Target : go away
Predicted : get away 

French(Source) : elle laide
Target : she helps him
Predicted : she helps him 

French(Source) : je ny parviens pas
Target : i cant do it
Predicted : i cant do it 

French(Source) : jai la priorite sur vous
Target : i outrank you
Predicted : i outrank you 

French(Source) : je men suis doute
Target : i thought so
Predicted : i though

In [51]:
# Testing the Model-2 test sequences
bleu_test_both_lstm = model_evaluation(both_lstm, eng_tokenizer, x_test, test)

French(Source) : faites comme il vous plaira
Target : do as you like
Predicted : do as you you 

French(Source) : nous partons a la retraite
Target : were retiring
Predicted : were starved 

French(Source) : tes givree
Target : youre nuts
Predicted : youre good 

French(Source) : apportez un appareil photo
Target : bring a camera
Predicted : bring a lawyer 

French(Source) : tu sembles contrariee
Target : you seem upset
Predicted : you look upset 

French(Source) : peutetre viendratelle
Target : she may come
Predicted : call the 

French(Source) : je dispose dun plan
Target : i have a plan
Predicted : i bought plan plan 

French(Source) : je dormis tout le jour
Target : i slept all day
Predicted : i all off 

French(Source) : jai vu cela
Target : i saw that
Predicted : i seen that 

French(Source) : tom sauta
Target : tom jumped
Predicted : tom yelled 

French(Source) : pouvonsnous taider
Target : can we help you
Predicted : can we help 

French(Source) : voila cinq dollars
Target : he

In [52]:
# Model 3:  Encoder GRU and Decoder LSTM cell

def gru_lstm(french_vocab, english_vocab, french_timesteps, english_timesteps, num_units):
	learning_rate = 1e-2
	model = Sequential()
	model.add(Embedding(french_vocab, num_units, input_length= french_timesteps, mask_zero=True))
	model.add(GRU(num_units))
	model.add(RepeatVector(english_timesteps))
	model.add(LSTM(num_units,return_sequences=True))
	model.add(TimeDistributed(Dense(english_vocab, activation='softmax')))
	return model

In [53]:
# Defining the Model-3

learning_rate = 1e-2
gru_lstm = gru_lstm(french_vocabulary_size, english_vocabulary_size, french_maximum_length, english_maximum_length, 256)
gru_lstm.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [54]:
# Summary of Model-3

print(gru_lstm.summary())
plot_model(gru_lstm, to_file='gru_lstm.png', show_shapes=True)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 256)           1472768   
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               394752    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 5, 256)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 256)            525312    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5, 2776)           713432    
Total params: 3,106,264
Trainable params: 3,106,264
Non-trainable params: 0
_________________________________________________________________
None
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz

In [55]:
# Fitting the Model-3

filename = 'gru_lstm.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
history_gru_lstm = gru_lstm.fit(x_train, y_train, epochs=30, batch_size=64, validation_data=(x_valid, y_valid), callbacks=[checkpoint], verbose=2)

Epoch 1/30
165/165 - 17s - loss: 3.5322 - categorical_accuracy: 0.4839 - val_loss: 3.0153 - val_categorical_accuracy: 0.5311

Epoch 00001: val_loss improved from inf to 3.01526, saving model to gru_lstm.h5
Epoch 2/30
165/165 - 11s - loss: 2.6284 - categorical_accuracy: 0.5694 - val_loss: 2.6154 - val_categorical_accuracy: 0.5872

Epoch 00002: val_loss improved from 3.01526 to 2.61542, saving model to gru_lstm.h5
Epoch 3/30
165/165 - 11s - loss: 2.1314 - categorical_accuracy: 0.6198 - val_loss: 2.4003 - val_categorical_accuracy: 0.6101

Epoch 00003: val_loss improved from 2.61542 to 2.40034, saving model to gru_lstm.h5
Epoch 4/30
165/165 - 11s - loss: 1.7506 - categorical_accuracy: 0.6601 - val_loss: 2.2451 - val_categorical_accuracy: 0.6261

Epoch 00004: val_loss improved from 2.40034 to 2.24515, saving model to gru_lstm.h5
Epoch 5/30
165/165 - 11s - loss: 1.4514 - categorical_accuracy: 0.6932 - val_loss: 2.1575 - val_categorical_accuracy: 0.6403

Epoch 00005: val_loss improved from 2.

In [56]:
# Saving the history of the Model-3

df_history_gru_lstm = pd.DataFrame(history_both_lstm.history)
df_history_gru_lstm.to_csv('df_history_gru_lstm.csv')
print(df_history_gru_lstm)

        loss  categorical_accuracy  val_loss  val_categorical_accuracy
0   3.612989              0.472171  3.123174                  0.506133
1   2.711535              0.549467  2.640196                  0.571200
2   2.177814              0.607276  2.416023                  0.597200
3   1.810572              0.643924  2.256093                  0.621467
4   1.500825              0.681695  2.188692                  0.632000
5   1.231123              0.718381  2.143722                  0.641067
6   1.018034              0.752762  2.159810                  0.652133
7   0.829502              0.788076  2.141968                  0.662000
8   0.675615              0.819867  2.167118                  0.660533
9   0.578506              0.843371  2.204639                  0.669067
10  0.491479              0.863543  2.238677                  0.667733
11  0.441413              0.876667  2.284345                  0.672133
12  0.387935              0.890076  2.273862                  0.670267
13  0.

In [57]:
# Loading the Model-3

gru_lstm = load_model('gru_lstm.h5')

In [58]:
# Model-3 on the training sequences

bleu_train_gru_lstm = model_evaluation(gru_lstm, eng_tokenizer, x_train, train)

French(Source) : je me suis marre
Target : i had some fun
Predicted : i had some fun 

French(Source) : laissezmoi sortir
Target : let me out
Predicted : let me out 

French(Source) : on peut me faire confiance
Target : im trustworthy
Predicted : we trustworthy win 

French(Source) : quelquun estil la
Target : anybody home
Predicted : anybody tom 

French(Source) : mettezle la
Target : put it there
Predicted : bring it there 

French(Source) : vous etes sournois
Target : youre sneaky
Predicted : youre sneaky 

French(Source) : ne vous precipitez pas
Target : dont rush
Predicted : dont fret 

French(Source) : degage
Target : go away
Predicted : beat away 

French(Source) : elle laide
Target : she helps him
Predicted : she helps him 

French(Source) : je ny parviens pas
Target : i cant do it
Predicted : i cant forget 

French(Source) : jai la priorite sur vous
Target : i outrank you
Predicted : i outrank you 

French(Source) : je men suis doute
Target : i thought so
Predicted : i thought

In [59]:
# Testing the Model-3 on the test sequences

bleu_test_gru_lstm = model_evaluation(gru_lstm, eng_tokenizer, x_test, test)

French(Source) : faites comme il vous plaira
Target : do as you like
Predicted : do as do do 

French(Source) : nous partons a la retraite
Target : were retiring
Predicted : were retired 

French(Source) : tes givree
Target : youre nuts
Predicted : youre you 

French(Source) : apportez un appareil photo
Target : bring a camera
Predicted : bring a camera 

French(Source) : tu sembles contrariee
Target : you seem upset
Predicted : you seem upset 

French(Source) : peutetre viendratelle
Target : she may come
Predicted : maybe is is 

French(Source) : je dispose dun plan
Target : i have a plan
Predicted : i have a 

French(Source) : je dormis tout le jour
Target : i slept all day
Predicted : i the the 

French(Source) : jai vu cela
Target : i saw that
Predicted : i saw that 

French(Source) : tom sauta
Target : tom jumped
Predicted : tom joined 

French(Source) : pouvonsnous taider
Target : can we help you
Predicted : can we help 

French(Source) : voila cinq dollars
Target : heres
Predict

In [60]:
# Model 4:  Encoder LSTM and Decoder GRU cell

def lstm_gru(french_vocab, english_vocab, french_timesteps, english_timesteps, num_units):
	learning_rate = 1e-2
	model = Sequential()
	model.add(Embedding(french_vocab, num_units, input_length= french_timesteps, mask_zero=True))
	model.add(LSTM(num_units))
	model.add(RepeatVector(english_timesteps))
	model.add(GRU(num_units,return_sequences=True))
	model.add(TimeDistributed(Dense(english_vocab, activation='softmax')))
	return model

In [61]:
# Defining the Model-4

learning_rate = 1e-2
lstm_gru = lstm_gru(french_vocabulary_size, english_vocabulary_size, french_maximum_length, english_maximum_length, 256)
lstm_gru.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [62]:
# Summary of Model-4

print(lstm_gru.summary())
plot_model(lstm_gru, to_file='lstm_gru.png', show_shapes=True)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 10, 256)           1472768   
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 5, 256)            0         
_________________________________________________________________
gru_3 (GRU)                  (None, 5, 256)            394752    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 5, 2776)           713432    
Total params: 3,106,264
Trainable params: 3,106,264
Non-trainable params: 0
_________________________________________________________________
None
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz

In [63]:
# Fitting the Model-4

filename = 'lstm_gru.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
history_lstm_gru = lstm_gru.fit(x_train, y_train, epochs=30, batch_size=64, validation_data=(x_valid, y_valid), callbacks=[checkpoint], verbose=2)

Epoch 1/30
165/165 - 16s - loss: 3.6424 - categorical_accuracy: 0.4711 - val_loss: 3.2285 - val_categorical_accuracy: 0.5009

Epoch 00001: val_loss improved from inf to 3.22854, saving model to lstm_gru.h5
Epoch 2/30
165/165 - 11s - loss: 2.8447 - categorical_accuracy: 0.5371 - val_loss: 2.7890 - val_categorical_accuracy: 0.5588

Epoch 00002: val_loss improved from 3.22854 to 2.78902, saving model to lstm_gru.h5
Epoch 3/30
165/165 - 12s - loss: 2.3612 - categorical_accuracy: 0.5900 - val_loss: 2.5039 - val_categorical_accuracy: 0.5945

Epoch 00003: val_loss improved from 2.78902 to 2.50387, saving model to lstm_gru.h5
Epoch 4/30
165/165 - 12s - loss: 1.9692 - categorical_accuracy: 0.6299 - val_loss: 2.3614 - val_categorical_accuracy: 0.6141

Epoch 00004: val_loss improved from 2.50387 to 2.36140, saving model to lstm_gru.h5
Epoch 5/30
165/165 - 12s - loss: 1.6437 - categorical_accuracy: 0.6668 - val_loss: 2.2189 - val_categorical_accuracy: 0.6277

Epoch 00005: val_loss improved from 2.

In [64]:
# Saving the history of the Model-4

df_history_lstm_gru = pd.DataFrame(history_both_lstm.history)
df_history_lstm_gru.to_csv('df_history_lstm_gru.csv')
print(df_history_lstm_gru)

        loss  categorical_accuracy  val_loss  val_categorical_accuracy
0   3.612989              0.472171  3.123174                  0.506133
1   2.711535              0.549467  2.640196                  0.571200
2   2.177814              0.607276  2.416023                  0.597200
3   1.810572              0.643924  2.256093                  0.621467
4   1.500825              0.681695  2.188692                  0.632000
5   1.231123              0.718381  2.143722                  0.641067
6   1.018034              0.752762  2.159810                  0.652133
7   0.829502              0.788076  2.141968                  0.662000
8   0.675615              0.819867  2.167118                  0.660533
9   0.578506              0.843371  2.204639                  0.669067
10  0.491479              0.863543  2.238677                  0.667733
11  0.441413              0.876667  2.284345                  0.672133
12  0.387935              0.890076  2.273862                  0.670267
13  0.

In [65]:
# Loading Model-4
lstm_gru = load_model('lstm_gru.h5')

In [66]:
# Model-4 on training sequences

bleu_train_lstm_gru = model_evaluation(lstm_gru, eng_tokenizer, x_train, train)

French(Source) : je me suis marre
Target : i had some fun
Predicted : i had for fun 

French(Source) : laissezmoi sortir
Target : let me out
Predicted : let me out 

French(Source) : on peut me faire confiance
Target : im trustworthy
Predicted : im trusted sorry 

French(Source) : quelquun estil la
Target : anybody home
Predicted : is anybody home 

French(Source) : mettezle la
Target : put it there
Predicted : put it there 

French(Source) : vous etes sournois
Target : youre sneaky
Predicted : youre sneaky 

French(Source) : ne vous precipitez pas
Target : dont rush
Predicted : dont rush 

French(Source) : degage
Target : go away
Predicted : go fly a 

French(Source) : elle laide
Target : she helps him
Predicted : she helps him 

French(Source) : je ny parviens pas
Target : i cant do it
Predicted : i cant buy it 

French(Source) : jai la priorite sur vous
Target : i outrank you
Predicted : i outrank you 

French(Source) : je men suis doute
Target : i thought so
Predicted : i went it 


In [67]:
# Testing Model-4 on test sequences

bleu_test_lstm_gru = model_evaluation(lstm_gru, eng_tokenizer, x_test, test)

French(Source) : faites comme il vous plaira
Target : do as you like
Predicted : do as you can 

French(Source) : nous partons a la retraite
Target : were retiring
Predicted : stay starving 

French(Source) : tes givree
Target : youre nuts
Predicted : youre you 

French(Source) : apportez un appareil photo
Target : bring a camera
Predicted : call a book 

French(Source) : tu sembles contrariee
Target : you seem upset
Predicted : you seem upset 

French(Source) : peutetre viendratelle
Target : she may come
Predicted : what liked 

French(Source) : je dispose dun plan
Target : i have a plan
Predicted : i have a plan 

French(Source) : je dormis tout le jour
Target : i slept all day
Predicted : i all every town 

French(Source) : jai vu cela
Target : i saw that
Predicted : i had that 

French(Source) : tom sauta
Target : tom jumped
Predicted : tom is 

French(Source) : pouvonsnous taider
Target : can we help you
Predicted : can we help 

French(Source) : voila cinq dollars
Target : heres
